
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Le projet a été développé à l'aide de Alice Breton, étudiante à la maîtrise en génie informatique. Elle a suivi le cours lors de la session Hiver 2019.



# Projet : Débordement d'égouts

La description du projet est disponible à l'adresse suivante :
https://www.kaggle.com/t/a238b752c33a41d9803c2cdde6bfc929

Ce calepin Jupyter de base permet de charger et de nettoyer les données fournies. La dernière section détaille la génération du fichier des prédictions afin de le soumettre sur Kaggle dans le bon format.

Dans un premier temps, vous devrez récupérer l'archive *data.zip* sur Moodle. Ce dossier contient les fichiers suivants :
- surverses.csv
- precipitation.csv
- ouvrages-surverses.csv
- test.csv

Veuillez le décompresser dans le répertoire de ce calepin.

Le fichier *surverse.csv* répertorie s'il y a surverse (1) ou non (0) au cours de la journée pour les 170 ouvrages de débordement de 2013 à 2018 pour les mois de mai à octobre (inclusivement). Des renseignements additionnels sur les données sont disponibles à l'adresse suivante :

http://donnees.ville.montreal.qc.ca/dataset/debordement


Le fichier *precipitation.csv* contient les précipitations horaires en dixième de *mm* enregistrées à 5 stations pluviométriques de 2013 à 2019 :
- McTavish (7024745)
- Ste-Anne-de-Bellevue (702FHL8)
- Montreal/Pierre Elliott Trudeau Intl (702S006)
- Montreal/St-Hubert (7027329)
- L’Assomption (7014160)

Plus d'informations sur les précipitations sont disponibles à l'adresse suivante :

https://climat.meteo.gc.ca/climate_data/hourly_data_f.html?hlyRange=2008-01-08%7C2019-11-12&dlyRange=2002-12-23%7C2019-11-12&mlyRange=%7C&StationID=30165&Prov=QC&urlExtension=_f.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=17&searchMethod=contains&Month=11&Day=12&txtStationName=montreal&timeframe=1&Year=2019

Le fichier *ouvrages-surverses.csv* contient différentes caractéristiques des ouvrages de débordement. 

http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse

Le fichier *test.csv* contient les ouvrages et les jours pour lesquels vous devez prédire s'il y a eu surverse (true) ou non (false). Notez que l'on s'intéresse ici à 5 ouvrages de débordement localisés tout autour de l'Ile de Montréal :
- 3260-01D dans Rivière-des-Prairies 
- 3350-07D dans Ahunstic 
- 4240-01D dans Pointe-aux-Trembles 
- 4350-01D dans le Vieux-Montréal 
- 4380-01D dans Verdun

#### Remarque

Dans le projet, on ne s'intéresse qu'aux surverses occasionnées par les précipitations. On ignore les surverses occasionnées par 
- fonte de neige (F)
- travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

On suppose que lorsqu'il n'y a pas de raison pour la surverse, il s'agit d'une surverse causée par les précipitations. Puisque Nous nous intéresserons uniquement aux surverses occasionnées par les précipitations liquides, nous ne considérons que les mois de mai à octobre inclusivement.

In [16]:
import Pkg; Pkg.add("Combinatorics")
using CSV, DataFrames, Statistics, Dates, Gadfly, Random, LinearAlgebra, GLM, Distributions, Combinatorics
include("functions.jl");

  Updating registry at `C:\Users\Mouns\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %>                                     ]  6.7 %3 % [============>                            ]  30.0 %]  46.7 %.3 % [===========================>             ]  66.7 %0 % Resolving package versions...
 Installed Combinatorics ─ v1.0.0
 Installed Polynomials ─── v0.6.0
  Updating `C:\Users\Mouns\.julia\environments\v1.2\Project.toml`
  [861a8166] + Combinatorics v1.0.0
  Updating `C:\Users\Mouns\.julia\environments\v1.2\Manifest.toml`
  [861a8166] + Combinatorics v1.0.0
  [f27b6e38] + Polynomials v0.6.0


┌ Info: Precompiling Combinatorics [861a8166-3701-5b0c-9a16-15d98fcdc6aa]
└ @ Base loading.jl:1242


# Chargement des données et nettoyage préliminaire

## Chargement et nettoyage des surverses et précipitations

In [17]:
data = CSV.read("data/surverses.csv",missingstring="-99999")
first(data,5)

# Extraction des surverses pour les mois de mai à octobre inclusivement
data = filter(row -> month(row.DATE) > 4, data) 
data = filter(row -> month(row.DATE) < 11, data) 

# Remplacement des valeurs *missing* dans la colonne :RAISON par "Inconnue"
raison = coalesce.(data[:,:RAISON],"Inconnue")
data[!,:RAISON] = raison

# Exlusion des surverses coccasionnées par d'autres facteurs que les précipitations liquides
data = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data) 
select!(data, [:NO_OUVRAGE, :DATE, :SURVERSE])

surverse_df = dropmissing(data, disallowmissing=true)

### CHARGEMENT DES PRÉCIPITATIONS
data = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data, Symbol("St-Hubert")=>:StHubert)

# Nettoyage des données sur les précipitations
data = filter(row -> month(row.date) > 4, data) 
data = filter(row -> month(row.date) < 11, data) 


,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,0,missing
2,2013-05-01,1,0,0,0,0,missing
3,2013-05-01,2,0,0,0,0,missing
4,2013-05-01,3,0,0,0,0,missing
5,2013-05-01,4,0,0,0,0,missing
6,2013-05-01,5,0,0,0,0,missing
7,2013-05-01,6,0,0,0,0,missing
8,2013-05-01,7,0,0,0,0,missing
9,2013-05-01,8,0,0,0,0,missing


In [18]:
# REMPLACER LES VALEURS MANQUANTES DANS LE DATAFRAME DE PRÉCIPITATIONS
# On veut remplacer les données manquante grâce à une simple régression linéaire (Ridge).
# Pour ce faire, chaque station météo agira comme variable explicative pour les 4 autres
# On drop les rangées dont les 5valeurs sont missing

data_full = deepcopy(data)
dropmissing!(data, disallowmissing=true)

models = Dict()

for i = 1:length(data_full[:, 1])
    if any(ismissing, data_full[i, 3:7]) #ismissing c est pas une variable mais une méthode de Julia, 3:7 pour les 5 stations
        missing = []
        notmissing = []
        for j=3:7 #on passe à travers les 5 colonnes pour regarder celles qui sont missing ou non
            if ismissing(data_full[i, j])
                push!(missing, names(data_full)[j]) #push généralisé à tous les conteneurs
                #Le point d exclamation pour dire à la fonction de modifier le conteneur qu on lui passe
            else
                push!(notmissing, names(data_full)[j])
            end
        end
        
        if length(notmissing) > 0 #si on a 5 missing sur la ligne on peut juste rien prédire donc on skip ça
            for m in missing #m c est la station à prédire
                if !haskey(models, (m, notmissing)) #ça véfirie si dans le dictionnaire models il y a la key (m, notmissing)
                    models[(m, notmissing)] = CoeffRidge(data, m, notmissing) #on change la case dans le dico palr le coef que la fonction nous retourne
                end
                data_full[i, m] = round(convert(Vector{Float64}, data_full[i, notmissing])' * models[(m, notmissing)]) # on ajoute à la case manque la nouvelle valeur = x*Beta
            end
        end
    end
end

dropmissing!(data_full)
data_full

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64,Int64
1,2013-05-01,0,0,0,0,0,0
2,2013-05-01,1,0,0,0,0,0
3,2013-05-01,2,0,0,0,0,0
4,2013-05-01,3,0,0,0,0,0
5,2013-05-01,4,0,0,0,0,0
6,2013-05-01,5,0,0,0,0,0
7,2013-05-01,6,0,0,0,0,0
8,2013-05-01,7,0,0,0,0,0
9,2013-05-01,8,0,0,0,0,0


## Extraction variables explicatives (Maude L.)

In [19]:
# Maximum de précipitations en 1 heure dans une journée.
X_pcp_max = by(data_full, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)

# Somme des précipitations dans une journée
X_pcp_sum = by(data_full, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)

# Précipitation à chaque heure pour chaque journée
X_pcp_hour = data_full

# Somme des précipitations des 2 heures précédentes

X_pcp_three_hours = copy(data_full)
buffer = copy(data_full)
for i=3:7
    for j=1:length(data_full[:,2])
        if (j > 2)
            X_pcp_three_hours[j, i] = (buffer[(j-2), i] + buffer[(j-1), i] + buffer[j,i])
        else
            X_pcp_three_hours[j, i] = buffer[j, i]
        end
    end
end

# Somme des précipitations des 2 jours précédents

X_pcp_two_days = copy(X_pcp_sum)
buffer = copy(X_pcp_sum)
for i=2:6
    for j=1:length(X_pcp_sum[:,2])
        if (j > 1)
            X_pcp_two_days[j, i] = buffer[(j-1), i] + buffer[j,i]
        else
            X_pcp_two_days[j, i] = buffer[j, i]
        end
    end
end


# Mois de la prédiction pour chaque ouvrage
X_date = DataFrame(NO_OUVRAGE=surverse_df[:,:NO_OUVRAGE], MOIS=zeros(Int64,length(surverse_df[:,1])), SURVERSE=surverse_df[:,:SURVERSE])
for i=1:length(surverse_df[:,1])
    X_date[i,:MOIS] = month(surverse_df[i,:DATE])
end

X_mois = DataFrame(MOIS=["Mai", "Juin", "Juillet", "Août", "Septembre", "Octobre"], RivierePrairie=zeros(Int64,6), Ahuntsic=zeros(Int64,6), PointeAuxTrembles=zeros(Int64,6), VieuxMontreal=zeros(Int64,6), Verdun=zeros(Int64,6))
for j=1:length(X_date[:,1])
    if(X_date[j,1] == "3260-01D")
        X_mois[(X_date[j,2]-4),:RivierePrairie] += (X_date[j,3])
    elseif (X_date[j,1] == "3350-07D")
        X_mois[(X_date[j,2]-4),:Ahuntsic] += (X_date[j,3])
    elseif (X_date[j,1] == "4240-01D")
        X_mois[(X_date[j,2]-4),:PointeAuxTrembles] += (X_date[j,3])
    elseif (X_date[j,1] == "4350-01D")
        X_mois[(X_date[j,2]-4),:VieuxMontreal] += (X_date[j,3])
    elseif (X_date[j,1] == "4380-01D")
        X_mois[(X_date[j,2]-4),:Verdun] += (X_date[j,3])
    end
end


In [20]:
# Création de 5 dataframes séparés pour les données de surverses des 5 ouvrages d'intérêt
ouvrage = ["3260-01D", "3350-07D", "4240-01D", "4350-01D", "4380-01D"]

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[1], surverse_df)
surverse_rivierePrairie = DataFrame(DATE=filtered_surverse[:, 2], RIVIEREPRAIRIE=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[2], surverse_df)
surverse_ahuntsic = DataFrame(DATE=filtered_surverse[:, 2], AHUNTSIC=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[3], surverse_df)
surverse_pointeAuxTrembles = DataFrame(DATE=filtered_surverse[:, 2], POINTEAUXTREMBLES=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[4], surverse_df)
surverse_vieuxMontreal = DataFrame(DATE=filtered_surverse[:, 2], VIEUXMONTREAL=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[5], surverse_df)
surverse_verdun = DataFrame(DATE=filtered_surverse[:, 2], VERDUN=filtered_surverse[:, 3]);


# Analyse exploratoire

Cette section consitue une analyse exploratoire superficielle permettant de voir s'il existe un lien entre les précipitations et les surverses.

Prenons arbitrairement l'ouvrage de débordement près du Bota-Bota (4350-01D). La station météorologique la plus proche est McTavish. Prenons deux variables explicatives simple :
- la somme journalière des précipitations
- le taux horaire maximum journalier de précipitations

#### Calcul de la quantité journalière de précipitations pour chacune des stations météorologiques

In [21]:
plot(df, x=:SURVERSE, y=:SUM, Geom.boxplot)

UndefVarError: UndefVarError: df not defined

# Modèle #1: Régression logistique 

### Écrire une description 

In [22]:
# Construction d'un dataframe contenant toutes les variables explicatives
#ouvrage = "3260-01D"

function CreateDataframe(ouvrage, surverse_df)
    df_train = filter(row -> row.NO_OUVRAGE == ouvrage, surverse_df)

    x₁ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière McTavish
    x₂ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière Bellevue
    x₃ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière Assomption
    x₄ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière St-Hubert
    x₅ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière PET
    x₆ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours McTavish
    x₇ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours Bellevue
    x₈ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours Assomption
    x₉ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours St-Hubert
    x₁₀ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours PET
    x₁₁ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier McTavish
    x₁₂ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier Bellevue
    x₁₃ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier Assomption
    x₁₄ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier St-Hubert
    x₁₅ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier PET



    for i=1:size(df_train,1)

        ind = findfirst(X_pcp_sum[:,:date] .== df_train[i,:DATE])
        
        x₁[i] = X_pcp_sum[ind,:McTavish]

        x₂[i] = X_pcp_sum[ind,:Bellevue]

        x₃[i] = X_pcp_sum[ind,:Assomption]

        x₄[i] = X_pcp_sum[ind,:StHubert]

        x₅[i] = X_pcp_sum[ind,:Trudeau]

        x₆[i] = X_pcp_two_days[ind, :McTavish]

        x₇[i] = X_pcp_two_days[ind, :Bellevue]

        x₈[i] = X_pcp_two_days[ind, :Assomption]

        x₉[i] = X_pcp_two_days[ind, :StHubert]

        x₁₀[i] = X_pcp_two_days[ind, :Trudeau]

        x₁₁[i] = X_pcp_max[ind, :McTavish]

        x₁₂[i] = X_pcp_max[ind, :Bellevue]

        x₁₃[i] = X_pcp_max[ind, :Assomption]

        x₁₄[i] = X_pcp_max[ind, :StHubert]

        x₁₅[i] = X_pcp_max[ind, :Trudeau]

    end

    df_train[!,:SUMMcTavish] = x₁
    df_train[!,:SUMBellevue] = x₂
    df_train[!,:SUMAssomption] = x₃
    df_train[!,:SUMStHubert] = x₄
    df_train[!,:SUMPET] = x₅
    df_train[!,:SUM2McTavish] = x₆
    df_train[!,:SUM2Bellevue] = x₇
    df_train[!,:SUM2Assomption] = x₈
    df_train[!,:SUM2StHubert] = x₉
    df_train[!,:SUM2PET] = x₁₀
    df_train[!,:MAXMcTavish] = x₁₁
    df_train[!,:MAXBellevue] = x₁₂
    df_train[!,:MAXAssomption] = x₁₃
    df_train[!,:MAXStHubert] = x₁₄
    df_train[!,:MAXPET] = x₁₅

    
    return df_train

end


CreateDataframe (generic function with 1 method)

In [28]:
function TrainLogit(df::DataFrame, var::Matrix{Float64})
#     X_var = df[:, filter(x -> (x in var), names(df))]
    X = hcat(ones(Float64, length(df[:, 1])), var)
    M = fit(GeneralizedLinearModel, X, df[:, :SURVERSE], Bernoulli(), LogitLink())
    
    return M
end

TrainLogit (generic function with 1 method)

In [29]:
function BestLogitModel()
    train, test = splitdataframe(CreateDataframe("3350-07D", surverse_df), 0.8)
    models = DataFrame(VARS=Array{Int64, 1}[], F₁=Float64[])
    
    for i=1:15
        comb = collect(combinations(collect(4:18), i))
        for columns in comb
            X_train = convert(Matrix{Float64}, train[:, columns])
            M = TrainLogit(train, X_train)
            X_test = hcat(ones(Float64, length(test[:, 1])), convert(Matrix{Float64}, test[:, columns]))
            θ = predict(M, X_test)
            Y = θ .> 0.5
            F₁ = computeF1score(convert(Array{Int64}, Y), test[:, :SURVERSE])
            push!(models, [columns, F₁])
        end
    end
    _, bestM = findmax(models[:, :F₁])
    return models[bestM, :]    
end 

BestLogitModel (generic function with 1 method)

In [30]:
bestModel = BestLogitModel()
println("Selected columns: $(bestModel[:VARS])")
println("F1: $(bestModel[:F₁])")

UndefVarError: UndefVarError: predict not defined

In [31]:
df_train_3260, df_test_3260 = splitdataframe(CreateDataframe("3260-01D", surverse_df), 0.7) 
df_train_3350, df_test_3350 = splitdataframe(CreateDataframe("3350-07D", surverse_df), 0.7)
df_train_4240, df_test_4240 = splitdataframe(CreateDataframe("4240-01D", surverse_df), 0.7)
df_train_4350, df_test_4350 = splitdataframe(CreateDataframe("4350-01D", surverse_df), 0.7)
df_train_4380, df_test_4380 = splitdataframe(CreateDataframe("4380-01D", surverse_df), 0.7)

M_3260 = TrainLogit(df_train_3260, [Symbol("SUMMcTavish"), Symbol("SUMBellevue"), Symbol("SUMAssomption"), Symbol("SUMStHubert"), Symbol("SUMPET"), Symbol("SUM2McTavish"), Symbol("SUM2Bellevue"), Symbol("SUM2Assomption"), Symbol("SUM2StHubert"), Symbol("SUM2PET"), Symbol("MAXMcTavish"), Symbol("")])
M_3350 = TrainLogit(df_train_3350, ) 
M_4240 = TrainLogit(df_train_4240, )
M_4350 = TrainLogit(df_train_4350, )
M_4380 = TrainLogit(df_train_4380, )

MethodError: MethodError: no method matching TrainLogit(::DataFrame, ::Array{Symbol,1})
Closest candidates are:
  TrainLogit(::DataFrame, !Matched::Array{Float64,2}) at In[28]:3

## Naive Bayes (Othman et Anis)

In [65]:
# import Pkg; Pkg.add("Conda")
# import Conda
# Conda.add("scikit-learn")
# using ScikitLearn
@sk_import naive_bayes: GaussianNB
@sk_import naive_bayes: BernoulliNB 
@sk_import naive_bayes: MultinomialNB
@sk_import metrics: accuracy_score
@sk_import metrics: f1_score



PyObject <function f1_score at 0x0000000001335F28>

In [66]:
train_df, test_df = splitdataframe(surverse_df, 0.7)

df_train = CreateDataframe("3260-01D", train_df)
df_test = CreateDataframe("3260-01D", test_df)

X_train = convert(Matrix, df_train)[:, 4:end]
y_train = convert(Matrix, df_train)[:, 3]

X_test = convert(Matrix, df_test)[:, 4:end]
y_test = convert(Matrix, df_test)[:, 3]

322-element Array{Any,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [106]:
gnb = GaussianNB()

PyObject GaussianNB(priors=None, var_smoothing=1e-09)

In [73]:
score_acc = accuracy_score(y_test, y_predicted) 
score_f1 = f1_score(y_test, y_predicted) 

println(score)
println(score_f1)

0.9044943820224719
0.6060606060606061


In [136]:
df_train, df_test = splitdataframe(CreateDataframe("3350-07D", surverse_df), 0.8)

(583×18 DataFrame. Omitted printing of 13 columns
│ Row │ NO_OUVRAGE │ DATE       │ SURVERSE │ SUMMcTavish │ SUMBellevue │
│     │ String     │ Date       │ Int64    │ Int64       │ Int64       │
├─────┼────────────┼────────────┼──────────┼─────────────┼─────────────┤
│ 1   │ 3350-07D   │ 2015-05-01 │ 0        │ 0           │ 0           │
│ 2   │ 3350-07D   │ 2015-05-02 │ 0        │ 0           │ 0           │
│ 3   │ 3350-07D   │ 2015-05-03 │ 0        │ 0           │ 0           │
│ 4   │ 3350-07D   │ 2015-05-05 │ 0        │ 2           │ 0           │
│ 5   │ 3350-07D   │ 2015-05-06 │ 0        │ 0           │ 0           │
│ 6   │ 3350-07D   │ 2015-05-07 │ 0        │ 0           │ 0           │
│ 7   │ 3350-07D   │ 2015-05-08 │ 0        │ 0           │ 0           │
│ 8   │ 3350-07D   │ 2015-05-09 │ 1        │ 50          │ 22          │
│ 9   │ 3350-07D   │ 2015-05-10 │ 0        │ 2           │ 0           │
│ 10  │ 3350-07D   │ 2015-05-12 │ 1        │ 88          │ 46          │
⋮

In [143]:
function BestNaiveModel()
    models = DataFrame(VARS=Array{Int64, 1}[], F₁=Float64[])    
    for i=1:15
        comb = collect(combinations(collect(4:18), i))
        for columns in comb
            y_test = convert(Matrix, df_test)[:, 3]
            y_train = convert(Matrix, df_train)[:, 3]
            X_train = convert(Matrix{Float64}, df_train[:, columns])
            X_test = convert(Matrix{Float64}, df_test[:, columns])
            gnb_model = gnb.fit(X_train, y_train)
            y_predicted = gnb_model.predict(X_test)
            F₁ = f1_score(y_test, y_predicted) 
#             println(columns, " F1:" , F₁)
            push!(models, [columns, F₁])
        end
    end
    _, bestM = findmax(models[:, :F₁])
    return models[bestM, :]    
end 

BestNaiveModel (generic function with 1 method)

In [144]:
best_naive = BestNaiveModel()
println("Selected columns: $(best_naive[:VARS])")
println("F1: $(best_naive[:F₁])")

(15,)
(105,)
(455,)
(1365,)
(3003,)
(5005,)
(6435,)
(6435,)
(5005,)
(3003,)
(1365,)
(455,)
(105,)
(15,)
(1,)
Selected columns: [4, 8, 11, 14, 18]
F1: 0.819672131147541


In [145]:
best_naive

,VARS,F₁
,Array…,Float64
2690,"[4, 8, 11, 14, 18]",0.819672


In [147]:
models = DataFrame(VARS=Array{Int64, 1}[], F₁=Float64[])    
for i=1:35 
    model = BestNaiveModel()
    push!(models, [model[1], model[2]])
end

(15,)
(105,)
(455,)
(1365,)
(3003,)
(5005,)
(6435,)
(6435,)
(5005,)
(3003,)
(1365,)
(455,)
(105,)
(15,)
(1,)
(15,)
(105,)
(455,)
(1365,)
(3003,)
(5005,)
(6435,)
(6435,)
(5005,)
(3003,)
(1365,)
(455,)
(105,)
(15,)
(1,)
(15,)
(105,)
(455,)
(1365,)
(3003

InterruptException: InterruptException:

In [76]:
threshold = 200

#X1: Somme de precipitation max de McTavish
ouvrage = "3260-01D"
df_train = filter(row -> row.NO_OUVRAGE == ouvrage, train_df)
x₁ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière McTavish
# println(df_train)

for i=1:size(df_train,1)
    ind = findfirst(X_pcp_sum[:,:date] .== df_train[i,:DATE])
    x₁[i] = X_pcp_sum[ind,:McTavish]
end
df_train[!,:SUMMcTavish] = x₁

println(df_train[1:10,:])

# # On get les jours ou y a surverse et non
# surverse_days = filter(row -> row.SURVERSE == 1, df_train)
# non_surverse_days = filter(row -> row.SURVERSE == 0, df_train)

# n₀ = length(non_surverse_days[:SURVERSE])
# n₁ = length(surverse_days[:SURVERSE])
# n = n₁ + n₀

# # nombre d'entrées sans surverses ou le max de preci est grand
# rainy_no_surverse = filter(row -> row.SUMMcTavish >= threshold, non_surverse_days)
# n₀₁ = length(rainy_no_surverse[:SURVERSE])

# # nombre d'entrées avec surverses ou le max de preci est grand
# rainy_surverse = filter(row -> row.SUMMcTavish >= threshold, surverse_days)
# n₁₁ = length(rainy_surverse[:SURVERSE])

# # nombre d'entrées sans surverses ou le max de preci est petit
# non_rainy_no_surverse = filter(row -> row.SUMMcTavish < threshold, non_surverse_days)
# n₀₀ = length(rainy_no_surverse[:SURVERSE])

# # nombre d'entrées avec surverses ou le max de preci est peti
# non_rainy_surverse = filter(row -> row.SUMMcTavish < threshold, surverse_days)
# n₁₀ = length(rainy_surverse[:SURVERSE])

# q_surverse = (n₁+1)/(n+2) * (n₁₁+1)/(n₁+2)
# q_normal = (n₀+1)/(n+2) * (n₀₁+1)/(n₀+2)

# # constante de normalisation
# c = q_surverse + q_normal

# p_surverse = q_surverse/c

10×4 DataFrame
│ Row │ NO_OUVRAGE │ DATE       │ SURVERSE │ SUMMcTavish │
│     │ String     │ Date       │ Int64    │ Int64       │
├─────┼────────────┼────────────┼──────────┼─────────────┤
│ 1   │ 3260-01D   │ 2013-05-01 │ 0        │ 0           │
│ 2   │ 3260-01D   │ 2013-05-02 │ 0        │ 0           │
│ 3   │ 3260-01D   │ 2013-05-04 │ 0        │ 0           │
│ 4   │ 3260-01D   │ 2013-05-06 │ 0        │ 0           │
│ 5   │ 3260-01D   │ 2013-05-08 │ 0        │ 0           │
│ 6   │ 3260-01D   │ 2013-05-11 │ 1        │ 106         │
│ 7   │ 3260-01D   │ 2013-05-12 │ 0        │ 0           │
│ 8   │ 3260-01D   │ 2013-05-14 │ 0        │ 0           │
│ 9   │ 3260-01D   │ 2013-05-15 │ 0        │ 40          │
│ 10  │ 3260-01D   │ 2013-05-16 │ 0        │ 0           │


# Création du fichier de prédictions pour soumettre sur Kaggle

Dans ce cas-ci, nous prédirons une surverse avec une probabilité de 1/2 sans considérer aucune variable explicative.

In [60]:
## Chargement du fichier de test
test = CSV.read("data/test.csv")
hcat(test, DataFrame(SURVERSE=zeros(Int64, length(test[:,1]))))

df_test = DataFrame(NO_OUVRAGE=test[:, :NO_OUVRAGE], DATE=test[:, :DATE], SUMMcTavish=zeros(Int64, length(test[:,1])),SUMBellevue=zeros(Int64, length(test[:,1])),SUMAssomption=zeros(Int64, length(test[:,1])),SUMStHubert=zeros(Int64, length(test[:,1])),SUMPET=zeros(Int64, length(test[:,1])),SUM2McTavish=zeros(Int64, length(test[:,1])),SUM2Bellevue=zeros(Int64, length(test[:,1])),SUM2Assomption=zeros(Int64, length(test[:,1])),SUM2StHubert=zeros(Int64, length(test[:,1])),SUM2PET=zeros(Int64, length(test[:,1])),MAXMcTavish=zeros(Int64, length(test[:,1])),MAXBellevue=zeros(Int64, length(test[:,1])),MAXAssomption=zeros(Int64, length(test[:,1])),MAXStHubert=zeros(Int64, length(test[:,1])),MAXPET=zeros(Int64, length(test[:,1])), SURVERSE=zeros(Float64, length(test[:,1])))
for i=1:length(df_test[:,1])
    row_sum = filter(row-> row.date== df_test[i,2], X_pcp_sum)
    row_sum_2_days = filter(row-> row.date== df_test[i,2], X_pcp_two_days)
    row_max = filter(row-> row.date== df_test[i,2], X_pcp_max)
    for j=2:6
        df_test[i, j+1] = row_sum[1, j]
    end
    for j=7:11
        df_test[i, j+1] = row_sum_2_days[1, j-5]
    end
    for j=12:16
        df_test[i, j+1] = row_max[1, j-10]
    end
end


M_3260 = TrainLogit("3260-01D", surverse_df)
M_3350 = TrainLogit("3350-07D", surverse_df)
M_4240 = TrainLogit("4240-01D", surverse_df)
M_4350 = TrainLogit("4350-01D", surverse_df)
M_4380 = TrainLogit("4380-01D", surverse_df)

df_3260 = filter(row -> row.NO_OUVRAGE == "3260-01D", df_test)
df_3350 = filter(row -> row.NO_OUVRAGE == "3350-07D", df_test)
df_4240 = filter(row -> row.NO_OUVRAGE == "4240-01D", df_test)
df_4350 = filter(row -> row.NO_OUVRAGE == "4350-01D", df_test)
df_4380 = filter(row -> row.NO_OUVRAGE == "4380-01D", df_test)


# Estimation de la probabilité de surverse de l'ensemble de test
X̃ = hcat(ones(Float64,length(df_test[:,1])),df_test[:,:SUMMcTavish], df_test[:,:SUMBellevue], df_test[:,:SUMAssomption], df_test[:,:SUMStHubert], df_test[:,:SUMPET], df_test[:, :SUM2McTavish], df_test[:, :SUM2Bellevue], df_test[:, :SUM2Assomption], df_test[:, :SUM2StHubert], df_test[:, :SUM2PET], df_test[:, :MAXMcTavish], df_test[:, :MAXBellevue], df_test[:, :MAXAssomption], df_test[:, :MAXStHubert], df_test[:, :MAXPET])
X̃_3260 = hcat(ones(Float64,length(df_3260[:,1])),df_3260[:,:SUMMcTavish], df_3260[:,:SUMBellevue], df_3260[:,:SUMAssomption], df_3260[:,:SUMStHubert], df_3260[:,:SUMPET], df_3260[:, :SUM2McTavish], df_3260[:, :SUM2Bellevue], df_3260[:, :SUM2Assomption], df_3260[:, :SUM2StHubert], df_3260[:, :SUM2PET], df_3260[:, :MAXMcTavish], df_3260[:, :MAXBellevue], df_3260[:, :MAXAssomption], df_3260[:, :MAXStHubert], df_3260[:, :MAXPET])
X̃_3350 = hcat(ones(Float64,length(df_3350[:,1])),df_3350[:,:SUMMcTavish], df_3350[:,:SUMBellevue], df_3350[:,:SUMAssomption], df_3350[:,:SUMStHubert], df_3350[:,:SUMPET], df_3350[:, :SUM2McTavish], df_3350[:, :SUM2Bellevue], df_3350[:, :SUM2Assomption], df_3350[:, :SUM2StHubert], df_3350[:, :SUM2PET], df_3350[:, :MAXMcTavish], df_3350[:, :MAXBellevue], df_3350[:, :MAXAssomption], df_3350[:, :MAXStHubert], df_3350[:, :MAXPET])
X̃_4240 = hcat(ones(Float64,length(df_4240[:,1])),df_4240[:,:SUMMcTavish], df_4240[:,:SUMBellevue], df_4240[:,:SUMAssomption], df_4240[:,:SUMStHubert], df_4240[:,:SUMPET], df_4240[:, :SUM2McTavish], df_4240[:, :SUM2Bellevue], df_4240[:, :SUM2Assomption], df_4240[:, :SUM2StHubert], df_4240[:, :SUM2PET], df_4240[:, :MAXMcTavish], df_4240[:, :MAXBellevue], df_4240[:, :MAXAssomption], df_4240[:, :MAXStHubert], df_4240[:, :MAXPET])
X̃_4350 = hcat(ones(Float64,length(df_4350[:,1])),df_4350[:,:SUMMcTavish], df_4350[:,:SUMBellevue], df_4350[:,:SUMAssomption], df_4350[:,:SUMStHubert], df_4350[:,:SUMPET], df_4350[:, :SUM2McTavish], df_4350[:, :SUM2Bellevue], df_4350[:, :SUM2Assomption], df_4350[:, :SUM2StHubert], df_4350[:, :SUM2PET], df_4350[:, :MAXMcTavish], df_4350[:, :MAXBellevue], df_4350[:, :MAXAssomption], df_4350[:, :MAXStHubert], df_4350[:, :MAXPET])
X̃_4380 = hcat(ones(Float64,length(df_4380[:,1])),df_4380[:,:SUMMcTavish], df_4380[:,:SUMBellevue], df_4380[:,:SUMAssomption], df_4380[:,:SUMStHubert], df_4380[:,:SUMPET], df_4380[:, :SUM2McTavish], df_4380[:, :SUM2Bellevue], df_4380[:, :SUM2Assomption], df_4380[:, :SUM2StHubert], df_4380[:, :SUM2PET], df_4380[:, :MAXMcTavish], df_4380[:, :MAXBellevue], df_4380[:, :MAXAssomption], df_4380[:, :MAXStHubert], df_4380[:, :MAXPET])

df_3260.SURVERSE = predict(M_3260, X̃_3260)
df_3350.SURVERSE = predict(M_3350, X̃_3350)
df_4240.SURVERSE = predict(M_4240, X̃_4240)
df_4350.SURVERSE = predict(M_4350, X̃_4350)
df_4380.SURVERSE = predict(M_4380, X̃_4380)

Ỹ = DataFrame(DATE=df_test[:,:DATE], NO_OUVRAGE=df_test[:,:NO_OUVRAGE], SURVERSE=zeros(Int64,length(df_test[:,1])))
for i=1:length(df_test[:,1])
    if df_test[i, :NO_OUVRAGE] == "3260-01D"
        ind = findfirst(df_3260[:, :DATE] .== df_test[i, :DATE])
        df_test[i,:SURVERSE] = df_3260[ind, :SURVERSE]
    elseif df_test[i, :NO_OUVRAGE] == "3350-07D"
        ind = findfirst(df_3350[:, :DATE] .== df_test[i, :DATE])
        df_test[i,:SURVERSE] = df_3350[ind, :SURVERSE]
    elseif df_test[i, :NO_OUVRAGE] == "4240-01D"
        ind = findfirst(df_4240[:, :DATE] .== df_test[i, :DATE])
        df_test[i,:SURVERSE] = df_4240[ind, :SURVERSE]
    elseif df_test[i, :NO_OUVRAGE] == "4350-01D"
        ind = findfirst(df_4350[:, :DATE] .== df_test[i, :DATE])
        df_test[i,:SURVERSE] = df_4350[ind, :SURVERSE]
    elseif df_test[i, :NO_OUVRAGE] == "4380-01D"
        ind = findfirst(df_4380[:, :DATE] .== df_test[i, :DATE])
        df_test[i,:SURVERSE] = df_4380[ind, :SURVERSE]
    end
end
for i=1:length(df_test[:,1])
    if df_test[i,:SURVERSE] > .5
        Ỹ[i, :SURVERSE] = 1
    else
        Ỹ[i, :SURVERSE] = 0
    end
end

# Pour chacune des lignes du fichier test, comportant un ouvrage et une date, une prédiction est requise.
# Dans ce cas-ci, utilisons une prédiction les plus naîve. 
# On prédit avec une chance sur deux qu'il y ait surverse, sans utiliser de variables explicatives
#n = size(test,1)
#surverse = rand(n) .> .5


# Création du fichier sampleSubmission.csv pour soumettre sur Kaggle
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=Ỹ[:,:SURVERSE])
CSV.write("sampleSubmission_with_zero.csv",sampleSubmission)

# Vous pouvez par la suite déposer le fichier sampleSubmission.csv sur Kaggle.

"sampleSubmission_with_zero.csv"